In [35]:
import os 
import pandas as pd 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")

data = pd.read_csv("guillaume.txt", sep = ";")
data.info()

data.head()

data.isna().sum()

/var/folders/46/7rlt2f9x4szchbk_4z7ycdtc0000gn/T/ipykernel_4910/448400790.py:21: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("guillaume.txt", sep = ";")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646774 entries, 0 to 4646773
Data columns (total 23 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   ZIBZIN                    object
 1   IDAvisAutorisationCheque  object
 2   FlagImpaye                object
 3   Montant                   object
 4   DateTransaction           object
 5   CodeDecision              object
 6   VerifianceCPT1            object
 7   VerifianceCPT2            object
 8   VerifianceCPT3            object
 9   D2CB                      object
 10  ScoringFP1                object
 11  ScoringFP2                object
 12  ScoringFP3                object
 13  TauxImpNb_RB              object
 14  TauxImpNB_CPM             object
 15  EcartNumCheq              object
 16  NbrMagasin3J              object
 17  DiffDateTr1               object
 18  DiffDateTr2               object
 19  DiffDateTr3               object
 20  CA3TRetMtt                object
 21  CA3TR   

ZIBZIN                      0
IDAvisAutorisationCheque    0
FlagImpaye                  0
Montant                     0
DateTransaction             0
CodeDecision                0
VerifianceCPT1              0
VerifianceCPT2              0
VerifianceCPT3              0
D2CB                        0
ScoringFP1                  0
ScoringFP2                  0
ScoringFP3                  0
TauxImpNb_RB                0
TauxImpNB_CPM               0
EcartNumCheq                0
NbrMagasin3J                0
DiffDateTr1                 0
DiffDateTr2                 0
DiffDateTr3                 0
CA3TRetMtt                  0
CA3TR                       0
Heure                       0
dtype: int64

In [36]:
#recode type of variable 
var_quanti = ["Montant","VerifianceCPT1", "VerifianceCPT2", "VerifianceCPT3","D2CB","ScoringFP1","ScoringFP2","ScoringFP3","TauxImpNb_RB","TauxImpNB_CPM","EcartNumCheq","NbrMagasin3J","DiffDateTr1", "DiffDateTr2","DiffDateTr3","CA3TRetMtt","CA3TR","Heure"]

var_quali = ["FlagImpaye" , "IDAvisAutorisationCheque" , "CodeDecision"]

# split variable to have date and hour
data[['Date','Heure_split']] = data['DateTransaction'].astype(str).str.split(" " , expand=True)

In [37]:
for i in var_quanti :

    #data[i] = data[i].apply(pd.to_numeric, downcast = "float", errors = "coerce")
    data[i] = data[i].replace(",", ".", regex= True).astype(float).round(0)

for i in var_quali :
    
    data[i] = data[i].astype(object)
    

#look the number modality 
data.FlagImpaye.value_counts()

KeyboardInterrupt: 

In [ ]:
index = 0
for i in data['Date'] : 
    
    
    if i =="2017-09-01":
        break
    else:
        index += 1
        train = data.iloc[:index, :]


test = data.iloc[index:, :]


train.to_csv("C:/Users/Sam/Documents/GitHub/FouilleDonneeMassive/train.csv")
test.to_csv("C:/Users/Sam/Documents/GitHub/FouilleDonneeMassive/test.csv")

KeyError: 'Date'

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

"""
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in range(train.shape[1]):
    train.loc[:,i] = le.fit_transform(train.loc[:,i])
    
    test.loc[:,i] = le.fit_transform(test.loc[:,i])
"""

Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = Ytrain['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction"], axis = 1)
Ytest  = test.FlagImpaye


In [ ]:
from joblib import Parallel, delayed
import joblib

"""
sm = BorderlineSMOTE(random_state = 0)
#XBdSmote , YBdSmote = sm.fit_resample(Xtrain, Ytrain)

def echant(ech, X, Y):
    
    XBdSmote , YBdSmote = ech.fit_resample(X, Y)
    
    return XBdSmote , YBdSmote


result = Parallel(n_jobs=2)(delayed(echant)(sm,Xtrain, Ytrain) for i in range(1))
"""

names=[]
f1score_ =[]

models={'SVC': SVC(),
       'RandomForest': RandomForestClassifier(),
       'DecisionTree': DecisionTreeClassifier(),
       'Naïve Bayes': GaussianNB(), 
       'Neural Network': MLPClassifier(),
       'knn' : KNeighborsClassifier(),
       'LDA': LinearDiscriminantAnalysis(),
       'GradientBoosting' : GradientBoostingClassifier(), 
       }

In [ ]:
with joblib.parallel_backend('multiprocessing', n_jobs=5) as parallel:
    
    sm = BorderlineSMOTE(random_state = 0)
    XBdSmote , YBdSmote = sm.fit_resample(Xtrain, Ytrain)

    
    for name, model in models.items():
        name_model = model
        name_fit = name_model.fit(XBdSmote , YBdSmote)
        name_pred = name_fit.predict(Xtest)
        f1score = f1_score(Ytest,name_pred, average = "macro")
        names.append(name)
        f1score_.append(f1score)
        
score_df = pd.DataFrame(zip(names, f1score_))
score_df.columns = ["Nom", "Score"]


"""
def train_model(X, y,Xtest, name, model):
        
    name_model = model
    name_fit = name_model.fit(X,y)
    name_pred = name_fit.predict(Xtest)
    f1score = f1_score(Ytest,name_pred, average = "macro")
    names.append(name)
    f1score_.append(f1score)
    
    return names, f1score_

                               
Model = Parallel(n_jobs=8)(delayed(train_model)(result[0], result[1],Xtest,name, model) for name, model in models.items())


score_df = pd.DataFrame(zip(Model[0], Model[1]))
score_df.columns = ["Nom", "Score"]
"""
score_df.to_csv("res.csv")


"""
for name, model in models.items():
    name_model = model
    name_fit = name_model.fit(XBdSmote,YBdSmote)
    name_pred = name_fit.predict(Xtest)
    f1score = f1_score(Ytest,name_pred, average = "macro")
    names.append(name)
    f1score_.append(f1score)

score_df = pd.DataFrame(zip(names, f1score_))
score_df.columns = ["Nom", "Score"]

try: 
    score_df.to_csv('/Users/titouanhoude/Documents/GitHub/FouilleDonneeMassive/res.csv', index = False, sep=';', encoding='utf-8')
except:
    score_df.to_csv('C:/Users/Sam/Documents/GitHub/FouilleDonneeMassive/res.csv', index = False, sep=';', encoding='utf-8')
"""